## Import libs

In [4]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

In [ ]:
# @title
!pip install --quiet vllm scikit-learn tqdm
print("Required libraries unsloth, vllm, scikit-learn, and tqdm installed successfully.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.9/474.9 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.0/355.0 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 M

## Connect to drive for test dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load model and tokenizer

In [7]:
from unsloth import FastLanguageModel
from google.colab import userdata
import torch

hf_token = userdata.get('HF_API_KEY')

base_model_name = "speakleash/Bielik-4.5B-v3.0-Instruct"
load_from_checkpoint = False
checkpoint_dir = None # Not applicable since load_from_checkpoint is False
test_data_path = "drive/MyDrive/mipd_test.jsonl"

max_seq_length = 32768
MAX_NEW_TOKENS = 256
BATCH_SIZE = 2
load_in_4bit = True

print(f"base_model_name: {base_model_name}")
print(f"load_from_checkpoint: {load_from_checkpoint}")
print(f"checkpoint_dir: {checkpoint_dir}")
print(f"test_data_path: {test_data_path}")


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "speakleash/Bielik-4.5B-v3.0-Instruct",
    max_seq_length = 32768, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    token = hf_token,
)

base_model_name: speakleash/Bielik-4.5B-v3.0-Instruct
load_from_checkpoint: False
checkpoint_dir: None
test_data_path: drive/MyDrive/mipd_test.jsonl
==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from unsloth import FastLanguageModel

if load_from_checkpoint:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = checkpoint_dir,
        max_seq_length = max_seq_length,
        load_in_4bit = load_in_4bit,
        disable_log_stats = True,
    )
else:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = base_model_name,
        max_seq_length = max_seq_length,
        load_in_4bit = load_in_4bit,
        token = hf_token,
        disable_log_stats = True,
    )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


TimeoutError: Unsloth: HuggingFace seems to be down after trying for 120 seconds :(
Check https://status.huggingface.co/ for more details.
As a temporary measure, use modelscope with the same model name ie:
```
pip install modelscope
import os; os.environ['UNSLOTH_USE_MODELSCOPE'] = '1'
from unsloth import FastLanguageModel
model = FastLanguageModel.from_pretrained('unsloth/gpt-oss-20b')
```

## Load and prepare dataset

In [ ]:
import json
from datasets import load_dataset

# 1. Load the test dataset (already loaded in jCYTe7JJNed6, reconfirming)
dataset_test = load_dataset("json", data_files={'test': test_data_path})

# 3. Define a function named format_prompt
def format_prompt(example):
    # Combine instruction for system message and input for the user message
    system_instruction = example['instruction']
    user_message = example['input']

    # Construct the ChatML formatted prompt
    messages = [
        {"role": "system", "content": system_instruction},
        {"role": "user", "content": user_message},
    ]
    example['prompt'] = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # 5. Parse the output field into a Python list of strings (ground truth tags)
    try:
        # CLEANING STEP: Remove Markdown formatting
        clean_json = example['output'].replace("```json", "").replace("```", "").strip()
        example['tags'] = json.loads(clean_json)['discovered_techniques']
    except json.JSONDecodeError:
        # Handle cases where output might not be perfectly valid JSON (e.g., during training data prep)
        example['tags'] = [] # Assign empty list if parsing fails
        print(f"Warning: Could not parse output: {example['output']}")

    # 6. Return the modified example
    return example
small_test_dataset = dataset_test['test'].select(range(60))
# 7. Apply the format_prompt function to the loaded test dataset
original_columns = small_test_dataset.column_names
dataset_test_formatted = small_test_dataset.map(format_prompt, remove_columns=original_columns)

print("Formatted prompts and ground truth tags generated for the test dataset.")
print(dataset_test_formatted)

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Formatted prompts and ground truth tags generated for the test dataset.
Dataset({
    features: ['prompt', 'tags'],
    num_rows: 60
})


## Define evaluation function
Metric 1: Parsing Success Rate (Did it output valid JSON?).

Metric 2: Format Correction Rate (How many invalid jsons were recovered?)

Metric 3: Classification Performance (If parsable, how accurate?).


In [ ]:
import json
import re
from sklearn.metrics import f1_score

def evaluate_response(response_text: str, ground_truth_tags: list):
    """
    Evaluates response with support for Dict format {"discovered_techniques": []}
    and Markdown stripping.
    """
    parsed_tags = []
    parsing_status = 'Failed'

    # 0. Pre-processing: Strip Markdown (Crucial for Strict Success)
    # If we don't do this, valid JSON wrapped in ```json will fail strict parsing
    clean_text = response_text.replace("```json", "").replace("```", "").strip()

    # Attempt 1: Strict JSON parsing
    try:
        parsed_output = json.loads(clean_text)

        # CASE A: Output is the expected Dictionary
        if isinstance(parsed_output, dict):
            # Extract the specific key we trained on
            parsed_tags = parsed_output.get("discovered_techniques", [])
            # Check if the inner content is actually a list
            if not isinstance(parsed_tags, list):
                 # Try to force it if it's a string representation
                 parsed_tags = []
            parsing_status = 'Strict Success'

        # CASE B: Model outputted a raw List (unlikely but possible)
        elif isinstance(parsed_output, list):
            parsed_tags = parsed_output
            parsing_status = 'Strict Success'

        else:
            raise ValueError("Parsed output is not a Dict or List.")

    except (json.JSONDecodeError, ValueError):
        # Attempt 2: Regex-based correction
        # We look for the list explicitly
        match = re.search(r'\[(.*?)\]', clean_text, re.DOTALL)
        if match:
            extracted_content = f"[{match.group(1)}]"
            try:
                parsed_output_recovered = json.loads(extracted_content)
                if isinstance(parsed_output_recovered, list):
                    parsed_tags = parsed_output_recovered
                    parsing_status = 'Recovered'
            except (json.JSONDecodeError, ValueError):
                pass

    # --- F1 CALCULATION ---
    parsed_tags = [str(tag) for tag in parsed_tags if tag is not None]
    ground_truth_tags = [str(tag) for tag in ground_truth_tags if tag is not None]

    all_unique_tags = sorted(list(set(parsed_tags + ground_truth_tags)))

    if not all_unique_tags:
        f1 = 1.0
    elif not ground_truth_tags and parsed_tags:
        f1 = 0.0
    elif ground_truth_tags and not parsed_tags:
        f1 = 0.0
    else:
        y_true = [1 if tag in ground_truth_tags else 0 for tag in all_unique_tags]
        y_pred = [1 if tag in parsed_tags else 0 for tag in all_unique_tags]
        f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

    return {
        'parsing_status': parsing_status,
        'parsed_tags': parsed_tags,
        'f1_score': f1,
    }

## Infer and evaluate all answers

In [ ]:
from unsloth import FastLanguageModel
import torch
from tqdm import tqdm
import json

print(f"Loading Model with max_seq_length = {max_seq_length}...")

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# --- 2. SIMPLE INFERENCE LOOP (No Chunks!) ---
print("Starting Long-Context Inference...")
evaluation_results = []

for example in tqdm(dataset_test_formatted, desc="Processing"):
    prompt = example['prompt']
    ground_truth_tags = example['tags']

    # Tokenize
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate
    # Since we have a massive context, we just feed the whole thing in.
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens = 256,
            use_cache = True,
            do_sample = False,
            temperature = 0.0,
             # Unsloth handles padding automatically usually, but being explicit is safe
            pad_token_id = tokenizer.pad_token_id
        )

    # Decode
    # Slice off the input prompt
    generated_ids = output_ids[:, inputs.input_ids.shape[1]:]
    raw_output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # --- 3. STANDARD EVALUATION ---
    # Clean JSON
    if '}' in raw_output:
        raw_output = raw_output[:raw_output.find('}') + 1]

    # Evaluate
    result = evaluate_response(raw_output, ground_truth_tags)

    evaluation_results.append({
        'original_prompt_len': inputs.input_ids.shape[1], # Log length to verify it worked
        'ground_truth': ground_truth_tags,
        'predicted': result['parsed_tags'],
        'f1_score': result['f1_score'],
        'raw_output': raw_output
    })

    # CRITICAL: Clear cache after massive context to avoid OOM on next iteration
    torch.cuda.empty_cache()


print(f"Done. Processed {len(evaluation_results)} documents.")

Loading Model with max_seq_length = 32768...
Starting Long-Context Inference...


Processing: 100%|██████████| 60/60 [05:08<00:00,  5.15s/it]

Done. Processed 60 documents.


In [ ]:
# --- 4. REPORTING ---
print("\n" + "="*60)
print(f"INFERENCE REPORT: {len(evaluation_results)} documents")
print("="*60)

if evaluation_results:
    # 1. Calculate Global Metric
    avg_f1 = sum(r['f1_score'] for r in evaluation_results) / len(evaluation_results)
    print(f"\nGlobal Average F1 Score: {avg_f1:.4f}")

    # 2. Print Sample Table
    print("\n--- Sample Results (First 10) ---")
    print(f"{'F1 Score':<10} | {'Ground Truth':<30} | {'Predicted':<30}")
    print("-" * 80)

    for res in evaluation_results[:10]:
        # Truncate lists for cleaner printing
        gt_str = str(res['ground_truth'])
        pred_str = str(res['predicted'])
        gt_display = (gt_str[:27] + '..') if len(gt_str) > 27 else gt_str
        pred_display = (pred_str[:27] + '..') if len(pred_str) > 27 else pred_str

        print(f"{res['f1_score']:.4f}     | {gt_display:<30} | {pred_display:<30}")

    print("-" * 80)
else:
    print("No results generated.")


INFERENCE REPORT: 60 documents

Global Average F1 Score: 0.5343

--- Sample Results (First 10) ---
F1 Score   | Ground Truth                   | Predicted                     
--------------------------------------------------------------------------------
1.0000     | []                             | []                            
1.0000     | []                             | []                            
1.0000     | []                             | []                            
0.0000     | ['REFERENCE_ERROR']            | []                            
0.1111     | ['REFERENCE_ERROR', 'EMOTIO..  | ['WHATABOUTISM', 'STRAWMAN'.. 
0.0000     | []                             | ['CHERRY_PICKING', 'EMOTION.. 
1.0000     | []                             | []                            
0.1111     | ['REFERENCE_ERROR', 'EMOTIO..  | ['CHERRY_PICKING', 'STRAWMA.. 
0.0000     | []                             | ['WHATABOUTISM', 'CHERRY_PI.. 
0.0000     | []                             | ['C